Pobierz dane for formacie GeoJson z wybranego źródła 

 

Źródło 1 

https://www.data.gov.uk/ 

https://environment.data.gov.uk/catchment-planning/v/c3-plan 

Źródło 2 

https://geojson.io/#map=2/0/20 

Źródło 3 

dbfs:/databricks-datasets/nyctaxi/ 

 

Zadanie 1 

Stwórz notatnik w którym użyjesz poniższych funkcji: 

Policz odległość pomiędzy punktami, sprawdź czy dany punkt geograficzny znajduje się w poligonie. Jeśli dane nie mają poligonu to możesz go stworzyć.  

Jeżeli dane z wybranego źródła nie mają wystarczająco danych żeby wykonać wszystkie funkcje, użyj innego źródła. 

Użyj jednej z wybranych bibliotek GeoMesa lub Mosaic lub Sedona. 

ST_Area 

ST_Distance 

ST_Contains 

ST_Intersects 

ST_Within 

In [0]:
%pip install databricks-mosaic


Python interpreter will be restarted.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached importlib_metadata-8.5.0-py3-none-any.whl (26 kB)
  Using cached zipp-3.20.2-py3-none-any.whl (9.2 kB)
  Created wheel for keplergl: filename=keplergl-0.3.2-py2.py3-none-any.whl size=17922343 sha256=cdac70304a076dd78fe9b5491107989d2c743a86f1125e0b0fa675210684f068
  Stored in directory: /root/.cache/pip/wheels/54/ae/15/d61c54bd63becaa070c9590060ae071f60673f8a0e0badac3c
Successfully built keplergl
Python interpreter will be restarted.


In [0]:
from mosaic import enable_mosaic
from pyspark.sql import functions as F

enable_mosaic(spark)



            DEPRECATION WARNING: 
                Please use a Databricks:
                    - Photon-enabled Runtime for performance benefits
                    - Runtime ML for spatial AI benefits
                Mosaic will stop working on this cluster after v0.3.x.


## Załadowanie danych

In [0]:
from pyspark.sql.functions import explode, to_json, col, expr

polygon_df = (
    spark.read.option("multiline", "true").json("dbfs:/FileStore/tables/region.geojson")
    .select(explode("features").alias("feature"))
    .select(to_json(col("feature.geometry")).alias("geometry_json"))
    .withColumn("polygon_geom", expr("st_geomfromgeojson(geometry_json)"))
    .drop("geometry_json")
)

polygon_df.createOrReplaceTempView("region")
polygon_df.show()

+--------------------+
|        polygon_geom|
+--------------------+
|{3, 4326, [[[20.0...|
+--------------------+



In [0]:
from pyspark.sql import Row

point_df = spark.createDataFrame([
    Row(wkt_point="POINT(21.0122 52.2297)")
]).withColumn("point_geom", expr("st_geomfromwkt(wkt_point)"))

point_df.createOrReplaceTempView("location")
point_df.show()

+--------------------+--------------------+
|           wkt_point|          point_geom|
+--------------------+--------------------+
|POINT(21.0122 52....|{1, 0, [[[21.0122...|
+--------------------+--------------------+



In [0]:
spark.sql("""
SELECT st_contains(r.polygon_geom, l.point_geom) AS point_in_region
FROM region r CROSS JOIN location l
""").show()


+---------------+
|point_in_region|
+---------------+
|          false|
+---------------+



In [0]:
spark.sql("""
SELECT st_distance(r.polygon_geom, l.point_geom) AS distance_deg
FROM region r CROSS JOIN location l
""").show()

+-------------------+
|       distance_deg|
+-------------------+
|0.41443117956072995|
+-------------------+



In [0]:
spark.sql("""
SELECT st_area(polygon_geom) AS area_deg
FROM region
""").show()

+--------+
|area_deg|
+--------+
|     0.0|
+--------+



In [0]:
spark.sql("""
SELECT st_intersects(r.polygon_geom, l.point_geom) AS point_intersects_region
FROM region r CROSS JOIN location l
""").show()

+-----------------------+
|point_intersects_region|
+-----------------------+
|                  false|
+-----------------------+

